In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
import plotly.express as px
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from IPython.display import display
from math import sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.inspection import permutation_importance

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../dataset/cleaned_data.csv')
df.head()

,Area,Item,Year,Yield,avg_temp,Pesticides,avg_precipitation,group,yield_normalized
0,Angola,Cassava,1990,41177,24.12,64.0,1010,0,0.087702
1,Angola,Cassava,1991,40295,24.02,79.0,1010,0,0.085821
2,Angola,Cassava,1992,42295,23.96,23.0,1010,0,0.090086
3,Angola,Cassava,1993,42295,24.15,169.0,1010,0,0.090086
4,Angola,Cassava,1994,58596,24.04,25.5,1010,0,0.124847


### Linear Regression Model

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Define the features and target variable
features = ['avg_temp', 'Pesticides', 'avg_precipitation', 'Item']
X = df[features]
y = df['Yield']

# Create a column transformer to apply OneHotEncoder to the 'Item' column
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Item'])
    ],
    remainder='passthrough'
)

# Create a pipeline with the preprocessor and the LinearRegression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Fit the pipeline to the data
pipeline.fit(X, y)

# Get the feature names after one-hot encoding
ohe = preprocessor.named_transformers_['onehot']
feature_names = ohe.get_feature_names_out(input_features=['Item'])
other_features = ['avg_temp', 'Pesticides', 'avg_precipitation']
all_features = list(feature_names) + other_features

# Get the coefficients for the features from the model
coefficients = pipeline.named_steps['model'].coef_

# Print out the features with their coefficients
feature_coefficients = dict(zip(all_features, coefficients))
for feature, coef in feature_coefficients.items():
    print(f'{feature}: {coef}')

# Print out the intercept
intercept = pipeline.named_steps['model'].intercept_
print(f'Intercept: {intercept}')

Item_Cassava: 73691.38333381955
Item_Maize: -51269.26243665961
Item_Plantains and others: 36321.717322162825
Item_Potatoes: 110373.3098962319
Item_Rice, paddy: -42439.490406804936
Item_Sorghum: -65222.84743043374
Item_Soybeans: -70384.68987578891
Item_Sweet potatoes: 37017.510523288845
Item_Wheat: -60960.25673407328
Item_Yams: 32872.6258082854
avg_temp: -1881.8009102492074
Pesticides: 0.11303158925875323
avg_precipitation: -7.151592588650146
Intercept: 127913.37725739882


In [4]:
# This time with polynomial features
from sklearn.preprocessing import PolynomialFeatures

categorical_features = ['Item']
numerical_features = ['avg_temp', 'Pesticides', 'avg_precipitation']
features = numerical_features + categorical_features
X = df[features]
y = df['Yield']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a column transformer to apply OneHotEncoder to the 'Item' column
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

# Define polynomial features with interaction only
poly_features = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# Create a pipeline with the preprocessor, polynomial features, and a linear regression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('polynomial_features', poly_features),
    ('linear_regression', LinearRegression())
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict and evaluate on training data
y_train_pred = pipeline.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train =  np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Predict and evaluate on testing data
y_test_pred = pipeline.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test =  np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results for training data
print(f'Training Data - R-squared: {r2_train}')
print(f'Mean Squared Error: {mse_train}')
print(f'Root Mean Squared Error: {rmse_train}')
print(f'Mean Absolute Error: {mae_train}')

# Print the results for testing data
print(f'Testing Data - R-squared: {r2_test}')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')
print(f'Mean Absolute Error: {mae_test}')

Training Data - R-squared: 0.7117227321699506
Mean Squared Error: 1979179822.1676779
Root Mean Squared Error: 44487.97390495186
Mean Absolute Error: 27297.063750659217
Testing Data - R-squared: 0.6835850746893448
Mean Squared Error: 2234526332.996134
Root Mean Squared Error: 47270.77673358175
Mean Absolute Error: 28285.205242170956


In [9]:
# experiment with linear regression without applying the 

# Create a column transformer to apply OneHotEncoder to the 'Item' column
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Item'])
    ],
    remainder='passthrough'
)

# Create a pipeline with the preprocessor and the LinearRegression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Fit the pipeline to the data
pipeline.fit(X_train, y_train)

# Predict and evaluate on training data
y_train_pred = pipeline.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train =  np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred)

# Predict and evaluate on testing data
y_test_pred = pipeline.predict(X_test)
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test =  np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the results for training data
print(f'Training Data - R-squared: {r2_train}')
print(f'Mean Squared Error: {mse_train}')
print(f'Root Mean Squared Error: {rmse_train}')
print(f'Mean Absolute Error: {mae_train}')

print('test data start here \n')
# Print the results for testing data
print(f'Testing Data - R-squared: {r2_test}')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')
print(f'Mean Absolute Error: {mae_test}')

Training Data - R-squared: 0.6461088192621558
Mean Squared Error: 2429654927.1181436
Root Mean Squared Error: 49291.52997339547
Mean Absolute Error: 30832.38565185251
test data start here 

Testing Data - R-squared: 0.6199842590081104
Mean Squared Error: 2683676123.5764027
Root Mean Squared Error: 51804.2095159882
Mean Absolute Error: 31714.73449654645


Note: As you can see the result is better with polynomial 

### Data Preparation for Modeling

In [5]:
# Encoding Categorical Features
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

categorical_features = ['Item', 'Area']
numerical_features = ['Pesticides', 'avg_temp', 'avg_precipitation']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

X = df[categorical_features + numerical_features]
y = df['Yield']

# Splitting data
X_transformed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

std_scaler = StandardScaler(with_mean=False)
X_train_scale = std_scaler.fit_transform(X_train)
# Scale test data for evaluation
X_test_scale = std_scaler.transform(X_test)

### Random Forrest

In [6]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest (Supervised learning)
rf = RandomForestRegressor(max_depth=None, min_samples_split=2, n_estimators=300, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

# R-squared
rf_train_score = r2_score(y_train, y_train_pred)
rf_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
rf_train_mae = mean_absolute_error(y_train, y_train_pred)
rf_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
rf_train_mse = mean_squared_error(y_train, y_train_pred)
rf_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
rf_train_rmse = np.sqrt(rf_train_mse)
rf_test_rmse = np.sqrt(rf_test_mse)

# Printing the results
print(f"Random Forest - Training R-squared: {rf_train_score}, Testing R-squared: {rf_test_score}")
print(f"Random Forest - Training MAE: {rf_train_mae}, Testing MAE: {rf_test_mae}")
print(f"Random Forest - Training MSE: {rf_train_mse}, Testing MSE: {rf_test_mse}")
print(f"Random Forest - Training RMSE: {rf_train_rmse}, Testing RMSE: {rf_test_rmse}")

Random Forest - Training R-squared: 0.9966919170441194, Testing R-squared: 0.9800000535520749
Random Forest - Training MAE: 1797.9047465646977, Testing MAE: 4578.1348319488925
Random Forest - Training MSE: 22711783.990528207, Testing MSE: 141239882.8927693
Random Forest - Training RMSE: 4765.688196947866, Testing RMSE: 11884.438686482812


### Gradient Boosting

In [7]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(n_estimators=300, learning_rate=0.2, max_depth=10, random_state=42)
gb.fit(X_train, y_train)

# Predictions
y_train_pred = gb.predict(X_train)
y_test_pred = gb.predict(X_test)

# R-squared
gb_train_score = r2_score(y_train, y_train_pred)
gb_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
gb_train_mae = mean_absolute_error(y_train, y_train_pred)
gb_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
gb_train_mse = mean_squared_error(y_train, y_train_pred)
gb_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
gb_train_rmse = np.sqrt(gb_train_mse)
gb_test_rmse = np.sqrt(gb_test_mse)

# Printing the results
print(f"Gradient Boosting - Training R-squared: {gb_train_score}, Testing R-squared: {gb_test_score}")
print(f"Gradient Boosting - Training MAE: {gb_train_mae}, Testing MAE: {gb_test_mae}")
print(f"Gradient Boosting - Training MSE: {gb_train_mse}, Testing MSE: {gb_test_mse}")
print(f"Gradient Boosting - Training RMSE: {gb_train_rmse}, Testing RMSE: {gb_test_rmse}")

Gradient Boosting - Training R-squared: 0.9994296884108363, Testing R-squared: 0.9812698999654889
Gradient Boosting - Training MAE: 833.0961013549303, Testing MAE: 4334.7053504787555
Gradient Boosting - Training MSE: 3915498.43009692, Testing MSE: 132272210.94477764
Gradient Boosting - Training RMSE: 1978.7618426927784, Testing RMSE: 11500.965652708368


### Decision Tree:
not very good but only for benchmarking

In [8]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=5, random_state=42)
dt.fit(X_train, y_train)

# Predictions
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# R-squared
dt_train_score = r2_score(y_train, y_train_pred)
dt_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
dt_train_mae = mean_absolute_error(y_train, y_train_pred)
dt_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
dt_train_mse = mean_squared_error(y_train, y_train_pred)
dt_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
dt_train_rmse = np.sqrt(dt_train_mse)
dt_test_rmse = np.sqrt(dt_test_mse)

# Printing the results
print(f"Decision Tree - Training R-squared: {dt_train_score}, Testing R-squared: {dt_test_score}")
print(f"Decision Tree - Training MAE: {dt_train_mae}, Testing MAE: {dt_test_mae}")
print(f"Decision Tree - Training MSE: {dt_train_mse}, Testing MSE: {dt_test_mse}")
print(f"Decision Tree - Training RMSE: {dt_train_rmse}, Testing RMSE: {dt_test_rmse}")

Decision Tree - Training R-squared: 0.9964839344728202, Testing R-squared: 0.9697373468833944
Decision Tree - Training MAE: 1443.9063410370945, Testing MAE: 4889.498446601941
Decision Tree - Training MSE: 24139697.164453454, Testing MSE: 213715251.35545036
Decision Tree - Training RMSE: 4913.216580251013, Testing RMSE: 14619.003090342732


### K Nearest Neighbors
even though the k nearest neighbors are good for classification problem but here we use it as a benchmarking

In [11]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(algorithm= 'auto', n_neighbors=3, weights= 'distance')
knn.fit(X_train, y_train)

# Predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# R-squared
knn_train_score = r2_score(y_train, y_train_pred)
knn_test_score = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
knn_train_mae = mean_absolute_error(y_train, y_train_pred)
knn_test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
knn_train_mse = mean_squared_error(y_train, y_train_pred)
knn_test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
knn_train_rmse = np.sqrt(knn_train_mse)
knn_test_rmse = np.sqrt(knn_test_mse)

# Printing the results
print(f"K-Nearest Neighbors - Training R-squared: {knn_train_score}, Testing R-squared: {knn_test_score}")
print(f"K-Nearest Neighbors - Training MAE: {knn_train_mae}, Testing MAE: {knn_test_mae}")
print(f"K-Nearest Neighbors - Training MSE: {knn_train_mse}, Testing MSE: {knn_test_mse}")
print(f"K-Nearest Neighbors - Training RMSE: {knn_train_rmse}, Testing RMSE: {knn_test_rmse}")

K-Nearest Neighbors - Training R-squared: 0.9996322766703062, Testing R-squared: 0.5707533276549899
K-Nearest Neighbors - Training MAE: 148.01914611251374, Testing MAE: 28856.69775204791
K-Nearest Neighbors - Training MSE: 2524620.133070607, Testing MSE: 3031345603.4483376
K-Nearest Neighbors - Training RMSE: 1588.9053253956345, Testing RMSE: 55057.65708281036


### MLPRegressor - Neural Network

In [7]:
from sklearn.neural_network import MLPRegressor

nn_model = MLPRegressor(hidden_layer_sizes=(250), max_iter=1000, random_state=42)

# nn_model = MLPRegressor(
#     hidden_layer_sizes=(100,),  # One hidden layer with 100 neurons
#     activation='relu',          # 'relu' activation function
# activation function introduces the non-linearity to learn complex features
#     solver='adam',              # 'adam' solver for weight optimization
# adam optimizer 
#     max_iter=500,               # Set the number of iterations (epochs)
#     random_state=42             # Set the seed for reproducibility
# )

# Training the model
nn_model.fit(X_train, y_train)

# Predicting and evaluating the model
y_train_pred = nn_model.predict(X_train)
y_test_pred = nn_model.predict(X_test)

# R-squared
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Mean Absolute Error (MAE)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Mean Squared Error (MSE)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Root Mean Squared Error (RMSE)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

# Printing the results
print(f"Neural Network - Training R-squared: {train_r2}, Testing R-squared: {test_r2}")
print(f"Neural Network - Training MAE: {train_mae}, Testing MAE: {test_mae}")
print(f"Neural Network - Training MSE: {train_mse}, Testing MSE: {test_mse}")
print(f"Neural Network - Training RMSE: {train_rmse}, Testing RMSE: {test_rmse}")

Neural Network - Training R-squared: 0.7508395102526952, Testing R-squared: 0.7251055664532179
Neural Network - Training MAE: 26761.140587159905, Testing MAE: 27885.675719750343
Neural Network - Training MSE: 1710621921.392718, Testing MSE: 1941308078.1548567
Neural Network - Training RMSE: 41359.665392659044, Testing RMSE: 44060.27778118128
